In [6]:
import pandas as pd
import json
import ollama
import os

MODEL_NAME = "qwen3:latest"


# -------------------------------------------------------------------
# 1. FUNGSI MEMANGGIL OLLAMA VIA API ollama.chat()
# -------------------------------------------------------------------
def ask_ollama(prompt):
    print("\n[DEBUG] Memanggil Ollama via API...")

    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}]
        )
    except Exception as e:
        print("[ERROR] Ollama API Error:", str(e))
        return ""

    output = response["message"]["content"].strip()
    print("[DEBUG] Output AI diterima")
    return output


# -------------------------------------------------------------------
# 2. MENENTUKAN KATEGORI PAKAI AI
# -------------------------------------------------------------------
def determine_category(name, causes, symptoms):
    print(f"[DEBUG] Menentukan kategori untuk: {name}")

    prompt = f"""
Anda adalah Hair & Scalp Expert (pakar rambut dan kulit kepala).

Tentukan KATEGORI kondisi rambut berikut berdasarkan analisis penyebab dan gejala:

Nama penyakit : {name}
Penyebab : {causes}
Gejala : {symptoms}

KATEGORI yang tersedia (boleh lebih dari satu, pisahkan dengan koma):
- berminyak
- rontok
- ketombe
- kering
- rusak

ATURAN:
- Tidak boleh menjawab "campuran".
- Jika kombinasi, sebutkan eksplisit, misalnya:
  berminyak, ketombe
  rontok, ketombe
  berminyak, rontok, ketombe
- Jawaban hanya daftar kategori. Tidak boleh ada kalimat lain.
"""

    result = ask_ollama(prompt)
    print("[DEBUG] Output kategori AI:", result)
    return result


# -------------------------------------------------------------------
# 3. GENERATE FRAME JSON
# -------------------------------------------------------------------
def generate_frame(name, category, causes, symptoms, treatment):
    print(f"[DEBUG] Generate frame untuk: {name}")

    prompt = f"""
Anda adalah seorang pakar rambut (Hair Expert Specialist).
Analisis dan buat FRAME KNOWLEDGE untuk Hair Expert System berikut:

Nama penyakit: {name}
Kategori: {category}
Penyebab: {causes}
Gejala: {symptoms}
Treatment: {treatment}

Format output HARUS JSON valid:

{{
  "name": "",
  "category": "",
  "cause": "",
  "symptoms": "",
  "diagnosis": "",
  "treatment": ""
}}

Ketentuan:
- Tulis ulang semua secara ilmiah dan singkat.
- Diagnosis harus berisi analisis profesional.
- Output hanya JSON tanpa teks tambahan.
"""

    result = ask_ollama(prompt)
    return result


# -------------------------------------------------------------------
# 4. SIMPAN FILE JSON
# -------------------------------------------------------------------
def save_json(filename, content):
    safe_name = filename.replace("/", "_")
    print(f"[DEBUG] Menyimpan file: {safe_name}.json")

    with open(safe_name + ".json", "w", encoding="utf-8") as f:
        f.write(content)

    print(f"[SAVED] {safe_name}.json")


# -------------------------------------------------------------------
# 5. PROSES CSV
# -------------------------------------------------------------------
def process_csv(csv_path):
    print("[DEBUG] Membaca CSV...")

    try:
        df = pd.read_csv(csv_path, encoding="utf-8")
    except UnicodeDecodeError:
        print("[WARNING] UTF-8 gagal, mencoba latin1...")
        df = pd.read_csv(csv_path, encoding="latin1")

    print("[DEBUG] Kolom terbaca:", list(df.columns))
    print("[DEBUG] Jumlah baris:", len(df))

    if df.empty:
        print("[ERROR] CSV kosong!")
        return

    for idx, row in df.iterrows():
        print(f"\n========== MEMPROSES BARIS {idx+1} ==========")

        name = str(row["NAMA PENYAKIT"]).strip()
        causes = str(row["CAUSES"]).strip()
        symptoms = str(row["GEJALA"]).strip()
        treatment = str(row["TREATMENT"]).strip()

        # 1. Tentukan kategori
        category = determine_category(name, causes, symptoms)

        # 2. Generate JSON frame
        frame_json = generate_frame(name, category, causes, symptoms, treatment)

        # 3. Simpan ke file
        save_json(name, frame_json)


# -------------------------------------------------------------------
# 6. MAIN
# -------------------------------------------------------------------
if __name__ == "__main__":
    process_csv("HAIR-EXPERT.csv")


[DEBUG] Membaca CSV...
[WARNING] UTF-8 gagal, mencoba latin1...
[DEBUG] Kolom terbaca: ['KATEGORI', 'NAMA PENYAKIT', 'CAUSES', 'TREATMENT', 'GEJALA']
[DEBUG] Jumlah baris: 10

========== MEMPROSES BARIS 1 ==========
[DEBUG] Menentukan kategori untuk: Androgenic alopecia

[DEBUG] Memanggil Ollama via API...
[DEBUG] Output AI diterima
[DEBUG] Output kategori AI: <think>
Okay, let's tackle this query. The user wants me to categorize androgenic alopecia into the given categories: berminyak, rontok, ketombe, kering, rusak. First, I need to recall what androgenic alopecia is. It's a common type of hair loss, often genetic and hormonal, involving DHT.

The symptoms mentioned include progressive thinning, miniaturization of hair, and specific patterns in men and women. The user also noted that sometimes there's itching or pain. The categories are oily, hair loss, dandruff, dry, damaged.

So, androgenic alopecia is primarily a hair loss condition. The main issue is the shedding of hair, so "ron

In [3]:
import os
import json

# Direktori tempat semua file penyakit disimpan
INPUT_DIR = "rule-based/diseases/"
OUTPUT_FILE = "allfix_diseases.json"

def load_disease_files():
    combined = []

    # Loop semua file dalam folder diseases
    for filename in os.listdir(INPUT_DIR):
        if filename.endswith(".json"):
            filepath = os.path.join(INPUT_DIR, filename)

            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    data = json.load(f)

                # Ambil field yang diperlukan
                disease_entry = {
                    "name": data.get("name", None),
                    "causes": data.get("cause") or data.get("causes"),
                    "symptoms": data.get("symptoms", None),
                    "treatment": data.get("treatment", None)
                }

                combined.append(disease_entry)

            except Exception as e:
                print(f"⚠️ Error membaca {filename}: {e}")

    return combined


def save_output(data):
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    print(f"✅ File output berhasil dibuat: {OUTPUT_FILE}")


if __name__ == "__main__":
    diseases = load_disease_files()
    save_output(diseases)


✅ File output berhasil dibuat: allfix_diseases.json
